<a href="https://colab.research.google.com/github/ramya-rk02/Enigma-Encoder/blob/main/Enigma_encoder_f.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Enigma encoder
 
# Enigma Settings 
rotors = ("I","II","III")
reflector = "UKW-B"
ringSettings ="ABC"
ringPositions = "DEF" 
plugboard = "AT BS DE FM IR KN LZ OW PV XY"
# ---------------------------------------------------
 
def caesarShift(str, amount):
    output = ""
 
    for i in range(0,len(str)):
        c = str[i]
        code = ord(c)
        if ((code >= 65) and (code <= 90)):
            c = chr(((code - 65 + amount) % 26) + 65)
        output = output + c
    
    return output
 
def encode(plaintext):
  global rotors, reflector,ringSettings,ringPositions,plugboard
  #Enigma Rotors and reflectors
  r1 = "EKMFLGDQVZNTOWYHXUSPAIBRCJ"
  r1Notch = "Q"
  r2 = "AJDKSIRUXBLHWTMCQGZNPYFVOE"
  r2Notch = "E"
  r3 = "JPGVOUMFYQBENHZRDKASXLICTW"
  r3Notch = "V"
  r4 = "NZJHGRCXMYSWBOUFAIVLPEKQDT"
  r4Notch = "J"
  r5 = "FKQHTLXOCBJSPDZRAMEWNIUYGV"
  r5Notch = "Z" 
  
  rotorDict = {"I":r1,"II":r2,"III":r3,"IV":r4,"V":r5}
  rotorNotchDict = {"I":r1Notch,"II":r2Notch,"III":r3Notch,"IV":r4Notch,"V":r5Notch}  
  
  reflectorB = {"A":"Y","Y":"A","B":"R","R":"B","C":"U","U":"C","D":"H","H":"D","E":"Q","Q":"E","F":"S","S":"F","G":"L","L":"G","I":"P","P":"I","J":"X","X":"J","K":"N","N":"K","M":"O","O":"M","T":"Z","Z":"T","V":"W","W":"V"}
  reflectorC = {"A":"F","F":"A","B":"V","V":"B","C":"P","P":"C","D":"J","J":"D","E":"I","I":"E","G":"O","O":"G","H":"Y","Y":"H","K":"R","R":"K","L":"Z","Z":"L","M":"X","X":"M","N":"W","W":"N","Q":"T","T":"Q","S":"U","U":"S"}
  
  alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
  rANotch = False
  rBNotch = False
  rCNotch = False
  
  if reflector=="UKW-B":
    reflectorDict = reflectorB
  else:
    reflectorDict = reflectorC
  
  #A = Left,  B = Mid,  C=Right 
  rA = rotorDict[rotors[0]]
  rB = rotorDict[rotors[1]]
  rC = rotorDict[rotors[2]]
  rANotch = rotorNotchDict[rotors[0]]
  rBNotch = rotorNotchDict[rotors[1]]
  rCNotch = rotorNotchDict[rotors[2]]
  
  rALetter = ringPositions[0]
  rBLetter = ringPositions[1]
  rCLetter = ringPositions[2]
  
  rASetting = ringSettings[0]
  offsetASetting = alphabet.index(rASetting)
  rBSetting = ringSettings[1]
  offsetBSetting = alphabet.index(rBSetting)
  rCSetting = ringSettings[2]
  offsetCSetting = alphabet.index(rCSetting)
  
  rA = caesarShift(rA,offsetASetting)
  rB = caesarShift(rB,offsetBSetting)
  rC = caesarShift(rC,offsetCSetting)
  
  if offsetASetting>0:
    rA = rA[26-offsetASetting:] + rA[0:26-offsetASetting]
  if offsetBSetting>0:
    rB = rB[26-offsetBSetting:] + rB[0:26-offsetBSetting]
  if offsetCSetting>0:
    rC = rC[26-offsetCSetting:] + rC[0:26-offsetCSetting]
 
  ciphertext = ""
  
  #plugboard settings into a dictionary
  plugboardConnections = plugboard.upper().split(" ")
  plugboardDict = {}
  for pair in plugboardConnections:
    if len(pair)==2:
      plugboardDict[pair[0]] = pair[1]
      plugboardDict[pair[1]] = pair[0]
  
  plaintext = plaintext.upper()  
  for letter in plaintext:
    encryptedLetter = letter  
    
    if letter in alphabet:
      #Rotate Rotors - happens as soon as a key is pressed
      rotorTrigger = False
      #Third rotor rotates by 1 for every key being pressed
      if rCLetter == rCNotch:
        rotorTrigger = True 
      rCLetter = alphabet[(alphabet.index(rCLetter) + 1) % 26]
      #Check if rotorB needs to rotate
      if rotorTrigger:
        rotorTrigger = False
        if rBLetter == rotorBNotch:
          rotorTrigger = True 
        rBLetter = alphabet[(alphabet.index(rBLetter) + 1) % 26]
  
        #Check if rotorA needs to rotate
        if (rotorTrigger):
          rotorTrigger = False
          rALetter = alphabet[(alphabet.index(rALetter) + 1) % 26]
             
      else:
        
        if rBLetter == rBNotch:
          rBLetter = alphabet[(alphabet.index(rBLetter) + 1) % 26]
          rALetter = alphabet[(alphabet.index(rALetter) + 1) % 26]
        
      #plugboard encryption
      if letter in plugboardDict.keys():
        if plugboardDict[letter]!="":
          encryptedLetter = plugboardDict[letter]
    
      #Rotors & Reflector Encryption
      offsetA = alphabet.index(rALetter)
      offsetB = alphabet.index(rBLetter)
      offsetC = alphabet.index(rCLetter)
 
      # Wheel 3 Encryption
      pos = alphabet.index(encryptedLetter)
      let = rC[(pos + offsetC)%26]
      pos = alphabet.index(let)
      encryptedLetter = alphabet[(pos - offsetC +26)%26]
      
      # Wheel 2 Encryption
      pos = alphabet.index(encryptedLetter)
      let = rB[(pos + offsetB)%26]
      pos = alphabet.index(let)
      encryptedLetter = alphabet[(pos - offsetB +26)%26]
      
      # Wheel 1 Encryption
      pos = alphabet.index(encryptedLetter)
      let = rA[(pos + offsetA)%26]
      pos = alphabet.index(let)
      encryptedLetter = alphabet[(pos - offsetA +26)%26]
      
      # Reflector encryption
      if encryptedLetter in reflectorDict.keys():
        if reflectorDict[encryptedLetter]!="":
          encryptedLetter = reflectorDict[encryptedLetter]
      
      #again through the rotors 
      # Wheel 1 Encryption
      pos = alphabet.index(encryptedLetter)
      let = alphabet[(pos + offsetA)%26]
      pos = rA.index(let)
      encryptedLetter = alphabet[(pos - offsetA +26)%26] 
      
      # Wheel 2 Encryption
      pos = alphabet.index(encryptedLetter)
      let = alphabet[(pos + offsetB)%26]
      pos = rB.index(let)
      encryptedLetter = alphabet[(pos - offsetB +26)%26]
      
      # Wheel 3 Encryption
      pos = alphabet.index(encryptedLetter)
      let = alphabet[(pos + offsetC)%26]
      pos = rC.index(let)
      encryptedLetter = alphabet[(pos - offsetC +26)%26]
      
      #plugboard encryption
      if encryptedLetter in plugboardDict.keys():
        if plugboardDict[encryptedLetter]!="":
          encryptedLetter = plugboardDict[encryptedLetter]
 
    ciphertext = ciphertext + encryptedLetter
  
  return ciphertext
 
#Main Program 
print("  ***** Enigma Encoder *****")
print("")
plaintext = input("Enter text to encode or decode:\n")
ciphertext = encode(plaintext)
 
print("\nEncoded text: \n " + ciphertext)

  ***** Enigma Encoder *****

Enter text to encode or decode:
hello

Encoded text: 
 IRYCQ
